In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from PIL import Image
import os

In [2]:
metadata_file = 'HAM10000_metadata.csv'
data = pd.read_csv(metadata_file)

In [3]:
data.head(4)

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp


In [4]:
image_ids = data['image_id'].values
labels = data['dx'].values

In [5]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)


In [6]:
X_train_ids, X_test_ids, y_train, y_test = train_test_split(
    image_ids, labels, test_size=0.2, random_state=42)

In [7]:
image_size = (28, 28)  
num_classes = len(label_encoder.classes_)

In [8]:
def load_images(image_ids, image_dir):
    images = []
    for image_id in image_ids:
        # Load and preprocess each image
        image_path = os.path.join(image_dir, f"{image_id}.jpg")
        image = preprocess_image(image_path, image_size)
        images.append(image)
    return np.array(images)

def preprocess_image(image_path, image_size):
    # Implement your image preprocessing steps here
    # (e.g., resize, normalize, etc.)
    image = Image.open(image_path).convert('RGB')
    image = image.resize(image_size)
    image = np.array(image) / 255.0  # Normalize pixel values between 0 and 1
    return image

In [9]:
image_dir1 = r'C:\Users\nagap\Downloads\HAM10000_images_part_1'  
X_train = load_images(X_train_ids, image_dir1)
X_test = load_images(X_test_ids, image_dir1)



In [10]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [11]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
251/251 [==============================] - 8s 26ms/step - loss: 1.0156 - accuracy: 0.6687 - val_loss: 0.9174 - val_accuracy: 0.6755
Epoch 2/10
251/251 [==============================] - 6s 25ms/step - loss: 0.8832 - accuracy: 0.6832 - val_loss: 0.8686 - val_accuracy: 0.6805
Epoch 3/10
251/251 [==============================] - 6s 26ms/step - loss: 0.8472 - accuracy: 0.6928 - val_loss: 0.8326 - val_accuracy: 0.6850
Epoch 4/10
251/251 [==============================] - 6s 25ms/step - loss: 0.8125 - accuracy: 0.7057 - val_loss: 0.8057 - val_accuracy: 0.7089
Epoch 5/10
251/251 [==============================] - 6s 23ms/step - loss: 0.7590 - accuracy: 0.7188 - val_loss: 0.7631 - val_accuracy: 0.7189
Epoch 6/10
251/251 [==============================] - 6s 24ms/step - loss: 0.7254 - accuracy: 0.7303 - val_loss: 0.7563 - val_accuracy: 0.7204
Epoch 7/10
251/251 [==============================] - 6s 25ms/step - loss: 0.7105 - accuracy: 0.7360 - val_loss: 0.7432 - val_accuracy: 0.7229

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [14]:
%pip install pydot

Note: you may need to restart the kernel to use updated packages.
